In [1]:
import sys
sys.path.insert(0,'../scripts')
from pycap_for_PESTPP_MOU import prepare_MOU_files
from pathlib import Path
import zipfile
import pyemu



# First prepare the MOU files
In the `../scripts` directory the script called `pycap_for_PESTPP_MOU.py` has all the logic required to configure a directory in which to run PEST++ MOU for a given configuration. In the next cell, 5 variables must be specified. They are outlined here:
 -  `pump_lbound_fraction` is a lower bound on pumping, applied to each well used in the optimization. This takes the form of a fraction (typically between 0.0 and 1.0) to be multiplied by the baseline pumping rate to form a lower bound. *  
 -  `pump_ubound_fraction` is an upper bound on pumping, applied to each well used in the optimization. This takes the form of a fraction (typically around 1.0, but can be selected) to be multiplied by the baseline pumping rate to form a lower bound. 
 - `objectives` determined what the multiple objectives to be optimized for are. You can make more custom versions of this, but currently three options are supported:
    - `depletion_q` aims to _minimize_ depletion (in CFS) and _maximize_ agricultural pumping (in CFS)
    - `fish_q` aims to _maximize_ the probability of Brook trout presence (in %) and _maximize_ agricultural pumping (in CFS)
    - `fish_dollars` aims to _maximize_ the probability of Brook trout presence (in %) and _maximize_ agricultural receipts (in cashmoney $$$)
- `depletion_potential_threshold` is a threshold (in the range of 0.0 to 1.0) of depletion potential below which wells are not managed for optimization. In other words, if a well's depletion potential falls below the threshold, it is maintained at its original baseline pumping rate throughout the optimization process. A spatial analysis can help guide this decision - see notebook `01_Calculate_Depletion_Potential.ipynb` for guidance.
- `scenario_name` will be assigned to both the configuration file saving these choices (and used by `06_MOU_postprocess.ipynb` for visualization and further analysis) and used to name the folder in which it will run.  
    - the file goes in a subdirectory within this notebooks directory (`notebooks/configurations`) and is named `<scenario_name>.yml`


\* note that for the `fish_dollars` objectives set, there is a pumping threshold hard-coded that sets any pumping less than 70% of the baseline to 0.

In [2]:
run_name, run_path = prepare_MOU_files(pump_lbound_fraction=0.69,
                      pump_ubound_fraction=1.2,
                      objectives='fish_dollars', #'depletion_q','fish_q', or 'fish_dollars'
                      depletion_potential_threshold=0.01,
                      scenario_name='fish_cash_baseline')

wrote configuration to: configurations/fish_cash_baseline.yml
error parsing metadata from 'obsnme', continuing
noptmax:50, npar_adj:64, nnz_obs:2
ALL READY TO RUN:
 scenario: fish_cash_baseline
 directory: ../pycap_runs/pycap_pest/run_fish_cash_baseline


/Users/mnfienen/miniforge3/envs/pycap/lib/python3.11/site-packages/pyogrio/__init__.py:7: DeprecationWarning: The 'shapely.geos' module is deprecated, and will be removed in a future version. All attributes of 'shapely.geos' are available directly from the top-level 'shapely' namespace (since shapely 2.0.0).


# Now run PESTPP-MOU below - note, this might take up to 20 or 30 minutes

In [3]:
pyemu.os_utils.run(f"pestpp-mou {run_name}.pst /e",cwd=str(run_path))

./pestpp-mou fish_cash_baseline.pst /e


             pestpp-mou: multi-objective optimization under uncertainty

                   by the PEST++ development team

...processing command line: ' ./pestpp-mou fish_cash_baseline.pst /e'
...using external run manager


version: 5.2.24
binary compiled on Nov 12 2025 at 11:54:09

using control file: "fish_cash_baseline.pst"
in directory: "/Users/mnfienen/Documents/GIT/LPR_redux/LPR_pycap_opt/pycap_runs/pycap_pest/run_fish_cash_baseline"
on host: "IGSACB116LHMNF2"
on a(n) apple operating system
with release configuration
started at 01/13/26 08:38:02

processing control file fish_cash_baseline.pst
              starting external run manager ...


  ---  initializing MOEA process  ---  
...using 'nsga2' env selector
...using binary tournament mating pool selector
...saving generation specific populations and archives every nth generation 1
using all adjustable parameters as decision variables:  64
...number of decision variables:  64
...max ru

In [4]:
# zip up all the results
for cf in run_path.glob('*.csv'):
    if "_pop" in cf.name or "archive" in cf.name or "summary" in cf.name:
        with zipfile.ZipFile(cf.with_suffix('.csv.zip'), 'w', compression=zipfile.ZIP_DEFLATED, compresslevel=9) as ofp:
            ofp.write(cf, arcname=cf.name)
        cf.unlink()